In [2]:
from cnns_classes import t1_quantizedCNN, t2_quantizedCNN

In [ ]:
#https://www.kaggle.com/code/vmarkin/advatt
# carregar as bibliotecas básicas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline
torch.manual_seed(4)

In [4]:
batch_size = 512

train_dataset_output_name = "train_dataset.pkl"
test_dataset_output_name = "test_dataset.pkl"

t1_quantizations=[2,4,8]
t2_quantizations=[2,4,8]

device = torch.device('cpu')

In [5]:
class SatImgDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.transform = T.ToTensor()
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        x = self.transform(self.X[index])
        y = torch.FloatTensor(self.y[index])
        return {'x':x, 'y':y}

In [ ]:
#falta testar
from utils import utils
dataset_train = utils.read_object(train_dataset_output_name)
loader_train = DataLoader(dataset_train, batch_size, shuffle=True)
del dataset_train


dataset_test = utils.read_object(test_dataset_output_name)
loader_test = DataLoader(dataset_test, batch_size, shuffle=False)
del dataset_test

In [7]:
def get_model_output_filename (topology,quant):
    return f"./pytorch_models/sat6-cnn-t{topology}w{quant}.pt"

In [8]:
def train_model(epochs,lr,topology,topology_class,quant):
    print(f"training t{topology}w{quant}")
    model = topology_class(bit_quantization=quant)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epochs = epochs
    criterion = nn.CrossEntropyLoss()
    model.train()
    i=0
    for e in range(epochs):
        if i % 5 == 0:
            print(f"epoch ({i})") 
        for batch in tqdm(loader_train):
            pred = model(batch['x'].to(device))
            loss = criterion(pred, batch['y'].to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        i+=1

    print("saving the pytorch model...")    
    torch.save(model.state_dict(), get_model_output_filename(topology=topology,quant=quant))
    print("finishing training")
    return get_model_output_filename(topology=topology,quant=quant)

In [9]:
def test_model(topology,topology_class,model_filename,quant):
    print(f"testing t{topology}w{quant}")
    model = topology_class(bit_quantization=quant)
    model.load_state_dict(torch.load(model_filename))
    with torch.no_grad():
        correct = 0
        i=0
        for batch in loader_test:
            pred = model(batch['x'].to(device))
            predicted = torch.max(pred, 1)[1]
            real_class = torch.max(batch['y'].to(device), 1)[1]
            correct += (predicted == real_class).sum()    
    accuracy = correct.item()/len(dataset_test)*100
    print(f"accuracy of this model: {accuracy}% ({len(dataset_test)} test cases)")
    print(f"finishing testing")

In [10]:
def make_model(topology_class,topology,quant):
    print(f"starting to generate pytorch model t{topology}w{quant}")
    model_filename = train_model(epochs=1,topology=topology,lr=3e-4,topology_class=topology_class,quant=quant)
    test_model(topology=topology,topology_class=topology_class,model_filename=model_filename,quant=quant)
    print(f"finishing model generate")
    print(f"------------------------")

In [11]:
for quant in t1_quantizations:
    make_model(topology_class=t1_quantizedCNN,topology=1,quant=quant)

for quant in t2_quantizations:
    make_model(topology_class=t2_quantizedCNN,topology=2,quant=quant)

starting to generate pytorch model t1w2
training t1w2
epoch (0)


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.78s/it]


saving the pytorch model...
finishing training
testing t1w2
accuracy of this model: 15.8203125% (512 test cases)
finishing testing
finishing model generate
------------------------
starting to generate pytorch model t1w4
training t1w4
epoch (0)


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.43s/it]


saving the pytorch model...
finishing training
testing t1w4
accuracy of this model: 3.7109375% (512 test cases)
finishing testing
finishing model generate
------------------------
starting to generate pytorch model t1w8
training t1w8
epoch (0)


...

saving the pytorch model...
finishing training
testing t2w2
accuracy of this model: 14.84375% (512 test cases)
finishing testing
finishing model generate
------------------------
starting to generate pytorch model t2w4
training t2w4
epoch (0)


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.50s/it]


saving the pytorch model...
finishing training
testing t2w4
accuracy of this model: 3.7109375% (512 test cases)
finishing testing
finishing model generate
------------------------
starting to generate pytorch model t2w8
training t2w8
epoch (0)


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


saving the pytorch model...
finishing training
testing t2w8
accuracy of this model: 18.75% (512 test cases)
finishing testing
finishing model generate
------------------------
